In [1]:
import os
import cv2

### Params


In [3]:
primary_label = "correct"
skip_frame_by_s = 0.25  # skip 0.25 second

label_list_dict = {
    "bench_press": [
        "correct",
        "chan_mat_can_bang",
        "co_tay_bi_uong",
        "mo_khuyu_tay_qua_rong",
        "ta_mat_can_bang",
        "ta_xuong_co",
    ],
    "dead_lift": [
        "correct",
        "chum_goi",
        "cong_lung",
        "cong_vai",
    ],
    "squat": [
        "correct",
        "got_chan_nhac_khoi_san",
        "hong_khong_doi_xung",
        "lung_cong",
        "mui_chan_huong_vao_nhau",
        "thanh_ta_dat_o_co",
        "vi_tri_chan_khong_doi_xung",
    ],
}

current_exercise = "bench_press"
current_label = "chan_mat_can_bang"

videos_source_folder = f"assets/{current_exercise}/{current_label}"
data_output_folder = f"data/{current_exercise}"
current_labels = label_list_dict[current_exercise]

### Kiểm tra thông số của video


In [4]:
# Convert datetime to string
def convert_datetime_to_string(datetime):
    return datetime.strftime("%Y%m%d_%H%M%S")

### Trích xuất video thành các ảnh( mỗi ảnh cách nhau 0.5s) và chuyển vào 1 folder


In [24]:
def extract_images_from_video(
    video_path: str, output_folder: str, time_step: float = 0.25
):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Lấy tên file gốc không bao gồm phần mở rộng
    base_name = os.path.splitext(os.path.basename(video_path))[0]
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(f"Frames per second using video.get(cv2.CAP_PROP_FPS): {fps}")

    frame_interval = int(fps * time_step)
    frame_count = 0
    saved_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            saved_count += 1
            # Định dạng tên file: <tên file gốc>_<số thứ tự dạng 3 chữ số>.jpg

            output_file = os.path.join(
                output_folder, f"{base_name}_{saved_count:03d}.jpg"
            )
            cv2.imwrite(output_file, frame)
        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()

In [25]:
import datetime
frames_extraction_folders = []

# Lấy danh sách các video trong thư mục
video_files = os.listdir(videos_source_folder)
for video_path in video_files:
    if not video_path.endswith(".mp4"):
        continue

    video_name = video_path.split(".")[0]
    current_date_time = convert_datetime_to_string(datetime.datetime.now())

    original_video_path = f"{videos_source_folder}/{video_path}"
    frames_extraction_path = (
        f"frames_extraction/{current_exercise}/{current_label}/{current_date_time}"
    )
    frames_extraction_folders.append(frames_extraction_path)

    print("-> Processing video: ", video_path)
    extract_images_from_video(
        original_video_path, frames_extraction_path, skip_frame_by_s
    )

-> Processing video:  20250119_150339.mp4
Frames per second using video.get(cv2.CAP_PROP_FPS): 30.000819284412394
-> Processing video:  20250119_150458.mp4
Frames per second using video.get(cv2.CAP_PROP_FPS): 30.00107458516535
-> Processing video:  20250119_150643.mp4
Frames per second using video.get(cv2.CAP_PROP_FPS): 30.001061765971965


In [26]:
# Các frame trích xuất được lưu trong các thư mục sau
frames_extraction_folders

['frames_extraction/bench_press/chan_mat_can_bang/20250121_221346',
 'frames_extraction/bench_press/chan_mat_can_bang/20250121_221551',
 'frames_extraction/bench_press/chan_mat_can_bang/20250121_221745']

### Tool


In [4]:
import sys
from PyQt5.QtWidgets import (
    QApplication,
    QLabel,
    QMainWindow,
    QPushButton,
    QWidget,
    QVBoxLayout,
    QHBoxLayout,
    QCheckBox,
)
from PyQt5.QtGui import QPixmap, QTransform
from PyQt5.QtCore import Qt
import os
import shutil
import csv

def copy_images_to_folder(input_file: str, output_folder: str, angle: int):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    pixmap = QPixmap(input_file)
    transform = QTransform().rotate(angle)
    rotated_pixmap = pixmap.transformed(transform)
    output_file = os.path.join(output_folder, os.path.basename(input_file))
    rotated_pixmap.save(output_file)

def window(input_folder: str, output_folder: str, label_list: list, primary_label: str):
    current_angle = 0

    app = QApplication(sys.argv)
    main_window = QMainWindow()
    main_window.setGeometry(500, 100, 1000, 800)
    main_window.setWindowTitle("Labeling Tool")
    main_window.setStyleSheet("background-color: black;")

    images = os.listdir(input_folder)
    image_count = len(images)
    current_image_index = 0

    csv_file = os.path.join(output_folder, "labels.csv")
    if not os.path.exists(csv_file):
        with open(csv_file, "w", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(["file_name", "label"])

    central_widget = QWidget()
    main_window.setCentralWidget(central_widget)

    layout = QVBoxLayout()
    central_widget.setLayout(layout)

    image_checkbox_layout = QHBoxLayout()
    layout.addLayout(image_checkbox_layout)

    name_pic = QLabel()
    name_pic.setStyleSheet("color: white;")
    layout.addWidget(name_pic)

    pixmap = QPixmap(f"{input_folder}/{images[current_image_index]}")
    label = QLabel()
    label.setPixmap(pixmap)
    label.setScaledContents(True)
    label.setFixedSize(500, 600)
    image_checkbox_layout.addWidget(label)

    checkbox_layout = QVBoxLayout()
    checkbox_layout.setContentsMargins(50, 0, 0, 0)
    image_checkbox_layout.addLayout(checkbox_layout)

    checkboxes = []
    for label_name in label_list:
        checkbox = QCheckBox(label_name)
        checkbox.setStyleSheet("color: white;")
        checkbox_layout.addWidget(checkbox)
        checkboxes.append(checkbox)

        def on_checkbox_state_changed(state, current_checkbox=checkbox):
            if current_checkbox.text() == primary_label and state == Qt.Checked:
                for other_checkbox in checkboxes:
                    if other_checkbox != current_checkbox:
                        other_checkbox.setChecked(False)
            if current_checkbox.text() != primary_label and state == Qt.Checked:
                for other_checkbox in checkboxes:
                    if other_checkbox.text() == primary_label:
                        other_checkbox.setChecked(False)

        checkbox.stateChanged.connect(on_checkbox_state_changed)

    label_status = QLabel()
    label_status.setStyleSheet("color: white;")
    layout.addWidget(label_status)

    def rotate_image(angle):
        nonlocal pixmap
        transform = QTransform().rotate(angle)
        rotated_pixmap = pixmap.transformed(transform)
        label.setPixmap(rotated_pixmap)

    def update_ui():
        nonlocal current_image_index, current_angle
        pixmap.load(f"{input_folder}/{images[current_image_index]}")
        label.setPixmap(pixmap)
        name_pic.setText(
            f"{current_image_index + 1}/{image_count} - {images[current_image_index]}"
        )
        rotate_image(current_angle)

    update_ui()

    button_layout = QHBoxLayout()
    layout.addLayout(button_layout)

    btn_back = QPushButton("Back")
    btn_back.setStyleSheet("background-color: white;")
    button_layout.addWidget(btn_back)

    def btn_back_clicked():
        nonlocal current_image_index
        current_image_index -= 1
        if current_image_index < 0:
            current_image_index = len(images) - 1
        for checkbox in checkboxes:
            checkbox.setChecked(False)
        update_ui()

    btn_back.clicked.connect(btn_back_clicked)

    btn_continue = QPushButton("Continue")
    btn_continue.setStyleSheet("background-color: white;")
    button_layout.addWidget(btn_continue)

    def btn_continue_clicked():
        nonlocal current_image_index
        current_image_index += 1
        if current_image_index >= len(images):
            current_image_index = 0
        for checkbox in checkboxes:
            checkbox.setChecked(False)
        update_ui()

    btn_continue.clicked.connect(btn_continue_clicked)

    btn_save = QPushButton("Save and continue (click S or s)")
    btn_save.setStyleSheet("background-color: white;")
    layout.addWidget(btn_save)

    def btn_save_clicked():
        nonlocal current_image_index
        selected_labels = [
            checkbox.text() for checkbox in checkboxes if checkbox.isChecked()
        ]
        if not selected_labels:
            label_status.setText("No label selected")
            return
        for label_name in selected_labels:
            copy_images_to_folder(
                f"{input_folder}/{images[current_image_index]}",
                f"{output_folder}/{label_name}",
                current_angle,
            )
        with open(csv_file, "a", newline="") as file:
            writer = csv.writer(file)
            for label_name in selected_labels:
                writer.writerow([images[current_image_index], label_name])
        label_status.setText(
            f"Copied {images[current_image_index]} to folders: {', '.join(selected_labels)}"
        )
        for checkbox in checkboxes:
            checkbox.setChecked(False)
        btn_continue_clicked()

    btn_save.clicked.connect(btn_save_clicked)

    def key_press_event(event):
        nonlocal current_image_index
        if event.key() in [Qt.Key_S, Qt.Key_S]:
            btn_save_clicked()

    main_window.keyPressEvent = key_press_event

    btn_rotate_left = QPushButton("Rotate Left")
    btn_rotate_left.setStyleSheet("background-color: white;")
    button_layout.addWidget(btn_rotate_left)

    def btn_rotate_left_clicked():
        nonlocal current_angle
        current_angle -= 90
        if current_angle < 0:
            current_angle = 270
        rotate_image(current_angle)

    btn_rotate_left.clicked.connect(btn_rotate_left_clicked)

    btn_rotate_right = QPushButton("Rotate Right")
    btn_rotate_right.setStyleSheet("background-color: white;")
    button_layout.addWidget(btn_rotate_right)

    def btn_rotate_right_clicked():
        nonlocal current_angle
        current_angle += 90
        if current_angle >= 360:
            current_angle = 0
        rotate_image(current_angle)

    btn_rotate_right.clicked.connect(btn_rotate_right_clicked)

    main_window.show()
    try:
        sys.exit(app.exec_())
    except:
        print("Exiting")


In [ ]:
# Chạy tool label trên các folder sau:
# Ví dụ: ['frames_extraction/bench_press/chan_mat_can_bang/20250121_221346']
frames_extraction_folders

#### Chú ý đổi tên human_name, vào trong folder để biết đang đánh nhãn cho ai

In [8]:
# Dựa vào đường dẫn của các folder trích xuất ảnh, xác định tên người tập
human_name = "tuan"
current_exercise = "bench_press"
current_label = "chan_mat_can_bang"

data_output_folder = f"data/{human_name}/{current_exercise}"
if not os.path.exists(data_output_folder):
    os.makedirs(data_output_folder)

#### Đôi khi kernel bị die nên cần gán thủ công input_folder 
Ví dụ: window(`'frames_extraction/bench_press/chan_mat_can_bang/20250121_221346'`, data_output_folder, current_labels, primary_label)

In [9]:
# Ví dụ: 'frames_extraction/bench_press/chan_mat_can_bang/20250121_221346'
window(frames_extraction_folders[0], data_output_folder, current_labels, primary_label)

Exiting
